In [4]:
# %%capture
# !pip install lightgbm shap imbalanced-learn

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import shap

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.inspection import permutation_importance

import os

In [5]:

# Configuration
# pd.set_option('display.max_columns', None)
# plt.style.use('seaborn')
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


In [6]:

# Load dataset (assuming SIF-2 is available as 'perovskites.csv')
df = pd.read_excel(os.path.join('..', 'Data', 'Supplementary Information File 2 (SIF-2).xlsx'))


In [7]:

# Data preprocessing
def preprocess_data(df):
    # Drop unnecessary columns
    df = df.drop(['Compound', 'A', 'B', 'In literature'], axis=1)
    
    # One-hot encode Valence A
    valence_a = pd.get_dummies(df['Valence A'], prefix='Valence_A')
    df = pd.concat([df, valence_a], axis=1)
    df = df.drop(['Valence A', 'Valence_A_4', 'Valence_A_5'], axis=1)
    
    # Convert target to categorical
    df['Lowest distortion'] = df['Lowest distortion'].astype('category')
    
    return df


In [8]:

processed_df = preprocess_data(df)
X = processed_df.drop('Lowest distortion', axis=1)
y = processed_df['Lowest distortion']

# Define features based on paper's final list
features = [
    'Valence_A_1', 'Valence_A_2', 'Valence_A_3',
    'Radius A at XII', 'Radius A at VI',
    'EN(A)', 'EN(B)',
    'Bond length A-O', 'Bond length B-O',
    'ΔENR', 'tG', 'τ', 'μ'
]


KeyError: 'Valence A'

In [ ]:

# Initialize LightGBM model
lgb_model = lgb.LGBMClassifier(
    objective='multiclass',
    class_weight='balanced',
    random_state=RANDOM_STATE,
    n_estimators=500,
    num_leaves=31,
    learning_rate=0.1
)

# Stratified K-Fold Cross Validation with SMOTE
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
smote = SMOTE(random_state=RANDOM_STATE)

cv_scores = []
for train_idx, test_idx in skf.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    # Apply SMOTE only to training data
    X_res, y_res = smote.fit_resample(X_train, y_train)
    
    # Train model
    lgb_model.fit(X_res, y_res)
    
    # Evaluate
    score = lgb_model.score(X_test, y_test)
    cv_scores.append(score)

print(f"Cross-validation Accuracy: {np.mean(cv_scores):.3f} ± {np.std(cv_scores):.3f}")


In [ ]:

# Full model training with SMOTE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
X_res, y_res = smote.fit_resample(X_train, y_train)
lgb_model.fit(X_res, y_res)

# Generate predictions
y_pred = lgb_model.predict(X_test)

# Performance metrics
print(classification_report(y_test, y_pred))


In [ ]:

# Confusion Matrix
def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=lgb_model.classes_,
                yticklabels=lgb_model.classes_)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

plot_confusion_matrix(y_test, y_pred)

# Feature Importance
def plot_feature_importance(model, features):
    importance = model.feature_importances_
    indices = np.argsort(importance)[::-1]
    
    plt.figure(figsize=(10,6))
    plt.title("Feature Importance")
    plt.bar(range(len(features)), importance[indices], align='center')
    plt.xticks(range(len(features)), np.array(features)[indices], rotation=90)
    plt.xlabel('Features')
    plt.ylabel('Importance')
    plt.show()

plot_feature_importance(lgb_model, features)

# SHAP Analysis
def shap_analysis(model, X_sample):
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_sample)
    
    plt.figure(figsize=(12,8))
    shap.summary_plot(shap_values, X_sample, plot_type="bar", class_names=model.classes_)
    plt.show()
    
    for i, class_name in enumerate(model.classes_):
        plt.figure(figsize=(12,6))
        shap.summary_plot(shap_values[i], X_sample, plot_type="dot", show=False)
        plt.title(f"SHAP Summary for {class_name}")
        plt.show()

shap_analysis(lgb_model, X.sample(100, random_state=RANDOM_STATE))  # Use subset for speed

# KDE Plots (assuming formation energy data is available)
def plot_kde(df, target_col='Formation energy'):
    plt.figure(figsize=(10,6))
    for crystal_class in df['Lowest distortion'].unique():
        subset = df[df['Lowest distortion'] == crystal_class]
        sns.kdeplot(subset[target_col], label=crystal_class)
    plt.xlabel('Formation Energy (eV)')
    plt.ylabel('Density')
    plt.title('KDE of Formation Energy by Crystal Structure')
    plt.legend()
    plt.show()

# Assuming formation energy data is available in original DF
plot_kde(df)